In [1]:
import pandas as pd
import os
import time
import fnmatch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
fixlen_path = '/home/jxin05/BBMAS_Data/fixlen_data/'

# this function split pure data samples into 70%/10%/20% as training, validation 
# and testing sets
def train_test_split(path):
    file_list = os.listdir(path)
    num_sample = len(file_list)
    idx = np.random.permutation(num_sample)
    train_idx = idx[:int(num_sample*0.7)]
    validate_idx = idx[int(num_sample*0.7) : int(num_sample*0.8)]
    test_idx = idx[int(num_sample*0.8):]
    trainfile = []
    validatefile = []
    testfile = []
    for i in train_idx:
        trainfile.append(file_list[i])
    for i in validate_idx:
        validatefile.append(file_list[i])
    for i in test_idx:
        testfile.append(file_list[i])
    return trainfile, validatefile, testfile

# load pure data and its label 
def load_original_data2(path,filelist, fix_len):
    X = np.zeros((len(filelist), 3, fix_len))
    Y = np.zeros((len(filelist),1))    
    for i in range(len(filelist)):
        x = remove_col(pd.read_csv(path+filelist[i])).values
        X[i,:,:] = x.T
    return X,Y

def label_fromList(Xlist):
    num_sample = len(Xlist)
    Y = np.ones((num_sample,1))
    X = np.zeros((num_sample, Xlist[0].shape[1], Xlist[0].shape[0]))
    for i in range(num_sample):
        X[i, :, :] = Xlist[i].T
    return X,Y

# remove some columns from raw .csv file
def remove_col(df):
    return df.drop(['EID','time','time_in_ms'], axis =1)

# returns a list of synthetic data with size 3x12000
# pick_dict is like {0: [10,21,13,44...], 1: [5,7,9,10...] } where each element 
# of the list (value of a key) is the index(from 3 to 58) of replaced part
def mix_data(path, filelist, fake_number, replace_time, replace_size = 200):
    pick_dict = {}
    mixed_data = []
    count = 0
    while(count < fake_number):
        pick = np.random.randint(0, len(filelist))
        pick_dict[count] = []
        target = pd.read_csv(path + filelist[pick])
        mix = np.asarray(remove_col(target).values)
        for j in range(replace_time):
            start = np.random.randint(3,len(target)/replace_size-1)
            while(start in pick_dict[count]):
                start = np.random.randint(3,len(target)/replace_size-1)
            pick_dict[count].append(start)
            replace_clip = take_from_others(path, filelist,replace_size, pick)
            mix[start*replace_size:(start+1)*replace_size, :] = replace_clip
        mixed_data.append(mix)
        count+=1
    return mixed_data, pick_dict 
 
# randomly take a clip from a sample
# pick is the index of data sample to be processed
def take_from_others(path, filelist, replace_size, pick):
    takefrom = np.random.randint(0,len(filelist))
    while(takefrom == pick):
        takefrom = np.random.randint(0,len(filelist))
    target_df = pd.read_csv(path+filelist[takefrom])
    start_idx = np.random.randint(0,len(target_df)-replace_size)
    replace_clip = target_df.iloc[start_idx:start_idx+replace_size]
    replace_clip = remove_col(replace_clip)
    return np.asarray(replace_clip.values)

# smooth boundary between pure signal and synthetic signal
# apply ExpMovingAverage on 20 data points with window 3
def smooth(mixed_data, pick_dict, window = 3):
    res = mixed_data
    for i in range(len(mixed_data)):
        for v in pick_dict[i]:
            #print(v)
            start = v*200-10
            end = v*200+10
            data_x = mixed_data[i][start:end,0]
            #print(data_x)
            data_y = mixed_data[i][start:end,1]
            data_z = mixed_data[i][start:end,2]
            smoothed_x = ExpMovingAverage(data_x, window)
            #print(smoothed_x)
            smoothed_y = ExpMovingAverage(data_y, window)
            smoothed_z = ExpMovingAverage(data_z, window)
            res[i][start+window+1:end-window,0] = smoothed_x[window+1:-window]
            #print(res[i][start+window+1:end-window,0])
            res[i][start+window+1:end-window,1] = smoothed_y[window+1:-window]
            res[i][start+window+1:end-window,2] = smoothed_z[window+1:-window]
    return res
def ExpMovingAverage(array, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    a = np.convolve(array, weights, mode='full')[:len(array)]
    a[:window] = a[window]
    return a

# reshape input data to fit network
def transformation(x):
    if(len(x.shape) == 3):
        inputslist = []
        for i in range(x.shape[0]):
            inputs_x = torch.from_numpy(x[i,0,:]).float()
            inputs_x = inputs_x.view(60,-1)
            inputs_y = torch.from_numpy(x[i,1,:]).float()
            inputs_y = inputs_y.view(60,-1)
            inputs_z = torch.from_numpy(x[i,2,:]).float()
            inputs_z = inputs_z.view(60,-1)
            inputs = torch.stack([inputs_x, inputs_y, inputs_z])
            inputs = inputs.transpose(0,1)
            inputslist.append(inputs)
        res = torch.stack(inputslist)
        res = res.view(x.shape[0], 60, 1, 3, 200)
        return res
    else:
        inputs_x = torch.from_numpy(x[0,:]).float()
        inputs_x = inputs_x.view(60,-1)
        inputs_y = torch.from_numpy(x[1,:]).float()
        inputs_y = inputs_y.view(60,-1)
        inputs_z = torch.from_numpy(x[2,:]).float()
        inputs_z = inputs_z.view(60,-1)
        inputs = torch.stack([inputs_x, inputs_y, inputs_z])
        inputs = inputs.transpose(0,1)
        inputs = inputs.view(-1, 60, 1, 3, 200)
        return inputs

# generate a batch 
def take_batch(batch_size, X, Y):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    for i in range(0,X.shape[0]-batch_size+1, batch_size):
        excerpt = indices[i:i + batch_size]
        yield X[excerpt], Y[excerpt]

def train_batch_m2one(net, trainX,trainY, valX, valY,
                      batch_size = 20, epochs = 80, lr = 0.006):
    opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss(weight= torch.tensor([1-(105/840), 105/840]))
    val_acc = 0
    best_model = None
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        train_loss = []
        net.train()
        for x,y in take_batch(batch_size, trainX, trainY):
            targets = torch.from_numpy(y).view(batch_size, -1)
            targets = targets.squeeze()
            #print(targets.size())
            h = tuple([each.data for each in h])
            opt.zero_grad()  
            output, h = net(x, h)
            output = output[:,:,-1]
            #print(output.size())
            loss = criterion(output, targets.long())
            train_loss.append(loss.item())
            loss.backward()
            opt.step()
        acc = check_acc(net, valX, valY)
        if(acc >= val_acc): # store the model performs best on validation set
            best_model = net
        print("Epoch: {}/{}...".format(e+1, epochs),
        "Train Loss: {:.4f}...".format(np.mean(train_loss)),
             "Validation acc: {:.4f}...".format(acc))
    final_model = net
    return best_model, final_model
        
def predict(net, x):
    with torch.no_grad():
        h = net.init_hidden(1)
        output, h = net(x,h)
    if(output[0,-1,0] >= output[0, -1, 1]):
        return 0
    else:
        return 1
    
def check_acc(net, X, Y):
    correct = 0
    wrong = 0
    for i in range(X.shape[0]):
        res = predict(net,X[i])
        if(Y[i] == res):
            correct += 1
        else:
            wrong += 1
    return correct/X.shape[0]
        
def onerun2(net, path):
    print("creating synthetic data")
    trainfile, validatefile, testfile = train_test_split(path)
    print(len(trainfile), len(validatefile), len(testfile))
    train_data1, train_dict1 = mix_data(path, trainfile, 210, 30) 
    train_data2, train_dict2 = mix_data(path, trainfile, 210, 20)
    train_data3, train_dict3 = mix_data(path, trainfile, 210, 15) 
    train_data4, train_dict4 = mix_data(path, trainfile, 210, 10)
    validate_data1, validate_dict1 = mix_data(path, validatefile, 30, 30)
    validate_data2, validate_dict2 = mix_data(path, validatefile, 30, 20)
    validate_data3, validate_dict3 = mix_data(path, validatefile, 30, 15)
    validate_data4, validate_dict4 = mix_data(path, validatefile, 30, 10) 
    test_data1, test_dict1 = mix_data(path, testfile, 60, 30) 
    test_data2, test_dict2 = mix_data(path, testfile, 60, 20)
    test_data3, test_dict3 = mix_data(path, testfile, 60, 15) 
    test_data4, test_dict4 = mix_data(path, testfile, 60, 10) 
    print("smoothing data")
    train_data1 = smooth(train_data1, train_dict1)
    train_data2 = smooth(train_data2, train_dict2)
    train_data3 = smooth(train_data3, train_dict3)
    train_data4 = smooth(train_data4, train_dict4)
    validate_data1 = smooth(validate_data1, validate_dict1)
    validate_data2 = smooth(validate_data2, validate_dict2)
    validate_data3 = smooth(validate_data3, validate_dict3)
    validate_data4 = smooth(validate_data4, validate_dict4)
    test_data1 = smooth(test_data1, test_dict1)
    test_data2 = smooth(test_data2, test_dict2)
    test_data3 = smooth(test_data3, test_dict3)
    test_data4 = smooth(test_data4, test_dict4)
    print("making datasets")
    X1_train, Y1_train = label_fromList(train_data1)
    X2_train, Y2_train = label_fromList(train_data2)
    X3_train, Y3_train = label_fromList(train_data3)
    X4_train, Y4_train = label_fromList(train_data4)
    original_trainX, original_trainY = load_original_data2(path,trainfile, 12000)
    
    X1_val, Y1_val = label_fromList(validate_data1)
    X2_val, Y2_val = label_fromList(validate_data2)
    X3_val, Y3_val = label_fromList(validate_data3)
    X4_val, Y4_val = label_fromList(validate_data4)
    original_valX, original_valY = load_original_data2(path,validatefile, 12000)
    
    X1_test, Y1_test = label_fromList(test_data1)
    X2_test, Y2_test = label_fromList(test_data2)
    X3_test, Y3_test = label_fromList(test_data3)
    X4_test, Y4_test = label_fromList(test_data4)
    original_testX, original_testY = load_original_data2(path,testfile, 12000)
    
    train_X = np.vstack([X1_train, X2_train, X3_train, original_trainX, X4_train])
    train_Y = np.vstack([Y1_train, Y2_train, Y3_train, original_trainY, Y4_train])
    test_X = np.vstack([X1_test, X2_test, X3_test, original_testX, X4_test])
    test_Y = np.vstack([Y1_test, Y2_test, Y3_test, original_testY, Y4_test])
    val_X = np.vstack([X1_val, X2_val, X3_val, original_valX, X4_val])
    val_Y = np.vstack([Y1_val, Y2_val, Y3_val, original_valY, Y4_val])
    best, final = train_batch_m2one(net, train_X, train_Y, val_X, val_Y)
    acc1 = check_acc(final, X1_test, Y1_test)
    acc2 = check_acc(final, X2_test, Y2_test)
    acc3 = check_acc(final, X3_test, Y3_test)
    acc4 = check_acc(final, X4_test, Y4_test)
    acc5 = check_acc(final, original_testX, original_testY)
    res_final = [acc1, acc2, acc3, acc4, acc5]
    
    acc1 = check_acc(best, X1_test, Y1_test)
    acc2 = check_acc(best, X2_test, Y2_test)
    acc3 = check_acc(best, X3_test, Y3_test)
    acc4 = check_acc(best, X4_test, Y4_test)
    acc5 = check_acc(best, original_testX, original_testY)
    res_best = [acc1, acc2, acc3, acc4, acc5]
    return res_best, res_final, best, final

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1,32,(2,8), stride = (1,4))
        self.conv2 = nn.Conv2d(32,64, (2,4), stride = (1,2))
        self.conv3 = nn.Conv1d(64,32, 4, stride = 2)
    
    def forward(self, x):
        inputs = self.conv1(x)
        inputs = F.relu(inputs)
        inputs = self.conv2(inputs)
        inputs = F.relu(inputs)
        inputs = inputs.squeeze()
        inputs = self.conv3(inputs)
        inputs = F.relu(inputs)
        
        return inputs

class cnn_lstm(nn.Module):
    def __init__(self, cnn = None, drop_prob = 0.3, n_class = 2, n_layer = 1):
        super(cnn_lstm, self).__init__()
        if(cnn == None):
            self.cnn = CNN()
        else:
            self.cnn = cnn
        self.n_class = n_class
        self.lstm = nn.LSTM(320, 128, num_layers = n_layer, batch_first = True)
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32,n_class)
        self.dropout = nn.Dropout(drop_prob)
        self.outlayer = nn.Softmax(dim = 2)
    
    def forward(self, x, hidden):
        inputs = transformation(x)
        batch_size, timesteps, C, H, W = inputs.size()
        inputs = inputs.view(batch_size*timesteps,C,H,W)
        c_out = self.cnn(inputs)
        lstm_in = c_out.view(batch_size, timesteps, -1)
        lstm_out, hidden = self.lstm(lstm_in,hidden)
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.outlayer(out)
        
        if(batch_size != 1):
            out = out.transpose(1,2)
        return out, hidden
    
    def init_hidden(self,batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(1, batch_size, 128).zero_(),
                      weight.new(1, batch_size, 128).zero_())
        return hidden

def init_weights(m):# used when no pre-trained CNN
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Conv2d or type(m) == nn.Conv1d or type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)
        
def init_weights2(m):# used when CNN is pre-trained 
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)

In [3]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.6961... Validation acc: 0.7037...
Epoch: 2/80... Train Loss: 0.6350... Validation acc: 0.6444...
Epoch: 3/80... Train Loss: 0.6186... Validation acc: 0.7481...
Epoch: 4/80... Train Loss: 0.5618... Validation acc: 0.9333...
Epoch: 5/80... Train Loss: 0.5283... Validation acc: 0.9704...
Epoch: 6/80... Train Loss: 0.5499... Validation acc: 0.7185...
Epoch: 7/80... Train Loss: 0.5063... Validation acc: 0.8074...
Epoch: 8/80... Train Loss: 0.5554... Validation acc: 0.9185...
Epoch: 9/80... Train Loss: 0.5393... Validation acc: 0.8000...
Epoch: 10/80... Train Loss: 0.5206... Validation acc: 0.9185...
Epoch: 11/80... Train Loss: 0.4687... Validation acc: 0.8296...
Epoch: 12/80... Train Loss: 0.5216... Validation acc: 0.9481...
Epoch: 13/80... Train Loss: 0.5433... Validation acc: 0.8296...
Epoch: 14/80... Train Loss: 0.4845... Validation acc: 0.8222...
Epoch: 15/80... Train Loss: 0.4358... Validation

In [4]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.6740... Validation acc: 0.8815...
Epoch: 2/80... Train Loss: 0.6225... Validation acc: 0.9037...
Epoch: 3/80... Train Loss: 0.5919... Validation acc: 0.7852...
Epoch: 4/80... Train Loss: 0.6180... Validation acc: 0.8296...
Epoch: 5/80... Train Loss: 0.5610... Validation acc: 0.8296...
Epoch: 6/80... Train Loss: 0.5625... Validation acc: 0.8741...
Epoch: 7/80... Train Loss: 0.5337... Validation acc: 0.9407...
Epoch: 8/80... Train Loss: 0.6117... Validation acc: 0.8444...
Epoch: 9/80... Train Loss: 0.5559... Validation acc: 0.8000...
Epoch: 10/80... Train Loss: 0.5206... Validation acc: 0.7852...
Epoch: 11/80... Train Loss: 0.5878... Validation acc: 0.8593...
Epoch: 12/80... Train Loss: 0.5709... Validation acc: 0.8296...
Epoch: 13/80... Train Loss: 0.5749... Validation acc: 0.8296...
Epoch: 14/80... Train Loss: 0.4952... Validation acc: 0.8889...
Epoch: 15/80... Train Loss: 0.5276... Validation

In [5]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.6795... Validation acc: 0.8889...
Epoch: 2/80... Train Loss: 0.6470... Validation acc: 0.7481...
Epoch: 3/80... Train Loss: 0.6108... Validation acc: 0.7556...
Epoch: 4/80... Train Loss: 0.5924... Validation acc: 0.7111...
Epoch: 5/80... Train Loss: 0.5503... Validation acc: 0.4667...
Epoch: 6/80... Train Loss: 0.5505... Validation acc: 0.6370...
Epoch: 7/80... Train Loss: 0.5406... Validation acc: 0.6444...
Epoch: 8/80... Train Loss: 0.4380... Validation acc: 0.9037...
Epoch: 9/80... Train Loss: 0.4732... Validation acc: 0.5630...
Epoch: 10/80... Train Loss: 0.4761... Validation acc: 0.7630...
Epoch: 11/80... Train Loss: 0.4994... Validation acc: 0.5926...
Epoch: 12/80... Train Loss: 0.5552... Validation acc: 0.9333...
Epoch: 13/80... Train Loss: 0.5213... Validation acc: 0.6074...
Epoch: 14/80... Train Loss: 0.4781... Validation acc: 0.6222...
Epoch: 15/80... Train Loss: 0.5401... Validation

In [6]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.6907... Validation acc: 0.8815...
Epoch: 2/80... Train Loss: 0.6550... Validation acc: 0.8741...
Epoch: 3/80... Train Loss: 0.5464... Validation acc: 0.8370...
Epoch: 4/80... Train Loss: 0.5084... Validation acc: 0.8444...
Epoch: 5/80... Train Loss: 0.4673... Validation acc: 0.9111...
Epoch: 6/80... Train Loss: 0.5481... Validation acc: 0.8889...
Epoch: 7/80... Train Loss: 0.5157... Validation acc: 0.9407...
Epoch: 8/80... Train Loss: 0.4265... Validation acc: 0.7704...
Epoch: 9/80... Train Loss: 0.4386... Validation acc: 0.8148...
Epoch: 10/80... Train Loss: 0.4224... Validation acc: 0.7778...
Epoch: 11/80... Train Loss: 0.4506... Validation acc: 0.9259...
Epoch: 12/80... Train Loss: 0.5166... Validation acc: 0.6963...
Epoch: 13/80... Train Loss: 0.5327... Validation acc: 0.7630...
Epoch: 14/80... Train Loss: 0.4743... Validation acc: 0.8815...
Epoch: 15/80... Train Loss: 0.5451... Validation

In [7]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.6993... Validation acc: 0.7926...
Epoch: 2/80... Train Loss: 0.6753... Validation acc: 0.8963...
Epoch: 3/80... Train Loss: 0.6764... Validation acc: 0.8370...
Epoch: 4/80... Train Loss: 0.6634... Validation acc: 0.9037...
Epoch: 5/80... Train Loss: 0.6432... Validation acc: 0.8370...
Epoch: 6/80... Train Loss: 0.6288... Validation acc: 0.7704...
Epoch: 7/80... Train Loss: 0.5996... Validation acc: 0.8667...
Epoch: 8/80... Train Loss: 0.6072... Validation acc: 0.8444...
Epoch: 9/80... Train Loss: 0.5490... Validation acc: 0.8222...
Epoch: 10/80... Train Loss: 0.5349... Validation acc: 0.7852...
Epoch: 11/80... Train Loss: 0.5499... Validation acc: 0.8074...
Epoch: 12/80... Train Loss: 0.5659... Validation acc: 0.7630...
Epoch: 13/80... Train Loss: 0.5712... Validation acc: 0.8222...
Epoch: 14/80... Train Loss: 0.5703... Validation acc: 0.9259...
Epoch: 15/80... Train Loss: 0.5641... Validation

In [8]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.6926... Validation acc: 0.8593...
Epoch: 2/80... Train Loss: 0.6528... Validation acc: 0.8593...
Epoch: 3/80... Train Loss: 0.5953... Validation acc: 0.7630...
Epoch: 4/80... Train Loss: 0.5529... Validation acc: 0.9407...
Epoch: 5/80... Train Loss: 0.5134... Validation acc: 0.8741...
Epoch: 6/80... Train Loss: 0.4992... Validation acc: 0.9630...
Epoch: 7/80... Train Loss: 0.6478... Validation acc: 0.8148...
Epoch: 8/80... Train Loss: 0.5957... Validation acc: 0.8370...
Epoch: 9/80... Train Loss: 0.5892... Validation acc: 0.8667...
Epoch: 10/80... Train Loss: 0.5519... Validation acc: 0.7704...
Epoch: 11/80... Train Loss: 0.5235... Validation acc: 0.9185...
Epoch: 12/80... Train Loss: 0.4913... Validation acc: 0.9185...
Epoch: 13/80... Train Loss: 0.5458... Validation acc: 0.7037...
Epoch: 14/80... Train Loss: 0.5789... Validation acc: 0.9185...
Epoch: 15/80... Train Loss: 0.4761... Validation

In [9]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.7008... Validation acc: 0.8741...
Epoch: 2/80... Train Loss: 0.6698... Validation acc: 0.8148...
Epoch: 3/80... Train Loss: 0.6592... Validation acc: 0.8370...
Epoch: 4/80... Train Loss: 0.6054... Validation acc: 0.9259...
Epoch: 5/80... Train Loss: 0.6054... Validation acc: 0.8519...
Epoch: 6/80... Train Loss: 0.5692... Validation acc: 0.6148...
Epoch: 7/80... Train Loss: 0.5201... Validation acc: 0.8519...
Epoch: 8/80... Train Loss: 0.5127... Validation acc: 0.8444...
Epoch: 9/80... Train Loss: 0.4956... Validation acc: 0.9407...
Epoch: 10/80... Train Loss: 0.4949... Validation acc: 0.9185...
Epoch: 11/80... Train Loss: 0.4629... Validation acc: 0.9333...
Epoch: 12/80... Train Loss: 0.5182... Validation acc: 0.6963...
Epoch: 13/80... Train Loss: 0.4916... Validation acc: 0.9037...
Epoch: 14/80... Train Loss: 0.5831... Validation acc: 0.6889...
Epoch: 15/80... Train Loss: 0.4850... Validation

In [10]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.7102... Validation acc: 0.8222...
Epoch: 2/80... Train Loss: 0.6502... Validation acc: 0.8593...
Epoch: 3/80... Train Loss: 0.6242... Validation acc: 0.9259...
Epoch: 4/80... Train Loss: 0.5986... Validation acc: 0.7926...
Epoch: 5/80... Train Loss: 0.5560... Validation acc: 0.9778...
Epoch: 6/80... Train Loss: 0.5364... Validation acc: 0.8889...
Epoch: 7/80... Train Loss: 0.5547... Validation acc: 0.8444...
Epoch: 8/80... Train Loss: 0.5211... Validation acc: 0.9704...
Epoch: 9/80... Train Loss: 0.5080... Validation acc: 0.9630...
Epoch: 10/80... Train Loss: 0.5286... Validation acc: 0.8667...
Epoch: 11/80... Train Loss: 0.4634... Validation acc: 0.9778...
Epoch: 12/80... Train Loss: 0.4531... Validation acc: 0.8370...
Epoch: 13/80... Train Loss: 0.4603... Validation acc: 0.9333...
Epoch: 14/80... Train Loss: 0.5213... Validation acc: 0.9407...
Epoch: 15/80... Train Loss: 0.4393... Validation

In [11]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.7043... Validation acc: 0.5481...
Epoch: 2/80... Train Loss: 0.6928... Validation acc: 0.7630...
Epoch: 3/80... Train Loss: 0.6514... Validation acc: 0.7926...
Epoch: 4/80... Train Loss: 0.5844... Validation acc: 0.7852...
Epoch: 5/80... Train Loss: 0.5607... Validation acc: 0.9481...
Epoch: 6/80... Train Loss: 0.5116... Validation acc: 0.9481...
Epoch: 7/80... Train Loss: 0.5118... Validation acc: 0.8963...
Epoch: 8/80... Train Loss: 0.5834... Validation acc: 0.8519...
Epoch: 9/80... Train Loss: 0.4815... Validation acc: 0.9185...
Epoch: 10/80... Train Loss: 0.5203... Validation acc: 0.9481...
Epoch: 11/80... Train Loss: 0.5039... Validation acc: 0.8741...
Epoch: 12/80... Train Loss: 0.4510... Validation acc: 0.8741...
Epoch: 13/80... Train Loss: 0.4662... Validation acc: 0.8222...
Epoch: 14/80... Train Loss: 0.5682... Validation acc: 0.8000...
Epoch: 15/80... Train Loss: 0.4953... Validation

In [12]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun2(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
105 15 30
smoothing data
making datasets
Epoch: 1/80... Train Loss: 0.6957... Validation acc: 0.8963...
Epoch: 2/80... Train Loss: 0.6371... Validation acc: 0.7778...
Epoch: 3/80... Train Loss: 0.5705... Validation acc: 0.8370...
Epoch: 4/80... Train Loss: 0.5681... Validation acc: 0.9556...
Epoch: 5/80... Train Loss: 0.5303... Validation acc: 0.9407...
Epoch: 6/80... Train Loss: 0.4834... Validation acc: 0.8370...
Epoch: 7/80... Train Loss: 0.4558... Validation acc: 0.9185...
Epoch: 8/80... Train Loss: 0.4533... Validation acc: 0.9852...
Epoch: 9/80... Train Loss: 0.5559... Validation acc: 0.8519...
Epoch: 10/80... Train Loss: 0.5482... Validation acc: 0.8593...
Epoch: 11/80... Train Loss: 0.5268... Validation acc: 0.8222...
Epoch: 12/80... Train Loss: 0.4976... Validation acc: 0.7259...
Epoch: 13/80... Train Loss: 0.4972... Validation acc: 0.9407...
Epoch: 14/80... Train Loss: 0.4694... Validation acc: 0.8815...
Epoch: 15/80... Train Loss: 0.5062... Validation